# �� Hyperparameter Tuning: RSF vs XGBoost-AFT

**Objetivo**: Encontrar la mejor configuración de hiperparámetros para modelos de supervivencia.

**Modelos**:
- Random Survival Forest (RSF)
- XGBoost-AFT (Accelerated Failure Time)

**Output**:
- `models/best_rsf.pkl`
- `models/best_xgb_aft.json`

---

In [ ]:
# ==============================================================================
# 0. CONFIGURACIÓN Y CARGA
# ==============================================================================
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from pathlib import Path

from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from sklearn.model_selection import GridSearchCV, StratifiedKFold, ParameterGrid

# ----------------------------
# Reproducibilidad básica
# ----------------------------
SEED = 42
np.random.seed(SEED)

# ----------------------------
# Paths y checks
# ----------------------------
# Usamos la versión corregida (v2) del dataset
DATA_PATH = Path("data/processed/train_survival_final.parquet")
MODELS_DIR = Path("models")
MODELS_DIR.mkdir(parents=True, exist_ok=True)

if not DATA_PATH.exists():
    raise FileNotFoundError(f"❌ No existe el archivo: {DATA_PATH.resolve()}")

# ----------------------------
# Cargar datos
# ----------------------------
df_train = pd.read_parquet(DATA_PATH)
print(f"✅ Datos cargados: {len(df_train)} registros | columnas: {df_train.shape[1]}")

required_cols = {"event", "duration"}
missing = required_cols - set(df_train.columns)
if missing:
    raise ValueError(f"❌ Faltan columnas requeridas: {missing}")

# Validación mínima
if (df_train["duration"] <= 0).any():
    bad = int((df_train["duration"] <= 0).sum())
    raise ValueError(f"❌ Hay {bad} filas con duration <= 0. Corrige antes de entrenar.")

print(f"   Duration range: [{df_train['duration'].min():.2f}, {df_train['duration'].max():.2f}]")
print(f"   Event rate: {df_train['event'].mean():.1%}")

In [ ]:
# ==============================================================================
# 1. PREPROCESAMIENTO PARA RSF
# ==============================================================================
cols_to_drop = ["event", "duration", "carrera_norm"]  # meta-data o labels
X_train = df_train.drop(columns=[c for c in cols_to_drop if c in df_train.columns])

# Si hay columnas no numéricas, intenta one-hot automáticamente
non_numeric_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric_cols:
    print(f"⚠️ Columnas no numéricas detectadas: {non_numeric_cols}")
    print("➡️ Aplicando one-hot encoding automático.")
    X_train = pd.get_dummies(X_train, columns=non_numeric_cols, drop_first=True)

# NaNs: imputación simple con mediana
if X_train.isna().any().any():
    print("⚠️ Se detectaron NaNs. Aplicando imputación (mediana).")
    X_train = X_train.fillna(X_train.median(numeric_only=True))

# Formato para Scikit-Survival (RSF)
y_sksurv = Surv.from_dataframe("event", "duration", df_train)

print(f"✅ Features (X): {X_train.shape[1]} columnas")
print(f"✅ Target (y): {len(y_sksurv)} registros estructurados")

---
## 2. Tuning Random Survival Forest (RSF)

Usamos StratifiedKFold para mantener proporción de eventos en cada fold.

In [ ]:
# ==============================================================================
# 2. TUNING RANDOM SURVIVAL FOREST (RSF)
# ==============================================================================
print("🌲 INICIANDO TUNING DE RSF (puede tardar varios minutos)...")

rsf_param_grid = {
    "n_estimators": [200, 500],
    "min_samples_leaf": [5, 10, 20],
    "max_depth": [None, 10],
    "max_features": ["sqrt"]
}

# Estratificar por event (proporción censura estable por fold)
y_event = df_train["event"].astype(int).values
cv_rsf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

rsf_grid = GridSearchCV(
    estimator=RandomSurvivalForest(random_state=SEED, n_jobs=-1),
    param_grid=rsf_param_grid,
    cv=cv_rsf,
    n_jobs=-1,
    verbose=1,
    refit=True
)

rsf_grid.fit(X_train, y_sksurv)

print(f"\n🏆 Mejor RSF C-index promedio: {rsf_grid.best_score_:.4f}")
print(f"⚙️ Mejores parámetros RSF: {rsf_grid.best_params_}")

# Guardar mejor modelo RSF
joblib.dump(rsf_grid.best_estimator_, MODELS_DIR / "best_rsf.pkl")
joblib.dump(rsf_grid.best_params_, MODELS_DIR / "best_rsf_params.pkl")
print(f"✅ Modelo RSF guardado en {MODELS_DIR / 'best_rsf.pkl'}")

---
## 3. Tuning XGBoost-AFT

XGBoost AFT requiere formato especial de censura:
- Evento=1 → tiempo exacto `[T, T]`
- Evento=0 → censura derecha → `[T, +inf]`

In [ ]:
# ==============================================================================
# 3. TUNING XGBOOST-AFT (Lógica avanzada de censura)
# ==============================================================================
print("🚀 INICIANDO TUNING DE XGBOOST-AFT...")

def make_aft_bounds(df):
    """
    AFT requiere intervalo [lower, upper]
    Evento=1 => tiempo exacto [T, T]
    Evento=0 => censura derecha => [T, +inf]
    """
    y_lower = df["duration"].to_numpy(dtype=float)
    y_upper = df["duration"].to_numpy(dtype=float)
    y_upper[df["event"].to_numpy(dtype=int) == 0] = np.inf
    return y_lower, y_upper

y_lower, y_upper = make_aft_bounds(df_train)

# DMatrix con bounds
dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info("label_lower_bound", y_lower)
dtrain.set_float_info("label_upper_bound", y_upper)

# Grid de hiperparámetros (reducido para tiempo razonable)
xgb_params_grid = {
    "learning_rate": [0.01, 0.05],
    "max_depth": [3, 5],
    "min_child_weight": [1, 3],
    "reg_lambda": [1.0, 5.0],
    "aft_loss_distribution": ["normal", "logistic"],
}

best_loss_val = float("inf")
best_params = None
best_num_rounds = None

grid = list(ParameterGrid(xgb_params_grid))
print(f"Evaluando {len(grid)} combinaciones para XGBoost...")

for i, params in enumerate(grid, start=1):
    current_params = dict(params)
    current_params.update({
        "objective": "survival:aft",
        "eval_metric": "aft-nloglik",
        "tree_method": "hist",
        "verbosity": 0,
        "seed": SEED,
        "nthread": -1
    })

    cv_results = xgb.cv(
        params=current_params,
        dtrain=dtrain,
        num_boost_round=1000,
        nfold=5,
        stratified=False,
        early_stopping_rounds=20,
        verbose_eval=False,
        seed=SEED
    )

    # ✅ BUG FIX: la mejor ronda es el argmin, no el total de filas
    loss_series = cv_results["test-aft-nloglik-mean"]
    mean_loss = float(loss_series.min())
    best_round_here = int(loss_series.idxmin() + 1)

    if mean_loss < best_loss_val:
        best_loss_val = mean_loss
        best_params = current_params
        best_num_rounds = best_round_here
        print(f"   🥇 Nuevo mejor ({i}/{len(grid)}): Loss={mean_loss:.4f} | "
              f"Dist={params['aft_loss_distribution']} | Rounds={best_num_rounds}")

print(f"\n🏆 Mejor XGBoost Loss (aft-nloglik): {best_loss_val:.4f}")
print(f"⚙️ Mejores parámetros XGB: {best_params}")
print(f"🔁 Mejor num_boost_round: {best_num_rounds}")

In [ ]:
# ==============================================================================
# 4. ENTRENAR MODELO FINAL XGBOOST-AFT
# ==============================================================================

# Entrenar el modelo con los mejores hiperparámetros y rondas óptimas
final_xgb = xgb.train(
    params=best_params,
    dtrain=dtrain,
    num_boost_round=best_num_rounds
)

# Guardar modelo
final_xgb.save_model(str(MODELS_DIR / "best_xgb_aft.json"))
joblib.dump(best_params, MODELS_DIR / "best_xgb_params.pkl")
print(f"✅ Modelo XGBoost-AFT guardado en {MODELS_DIR / 'best_xgb_aft.json'}")

In [ ]:
# ==============================================================================
# 📊 RESUMEN FINAL
# ==============================================================================
print("=" * 70)
print("📊 RESUMEN DEL HYPERPARAMETER TUNING")
print("=" * 70)

print(f"\n🌲 RANDOM SURVIVAL FOREST:")
print(f"   C-index promedio: {rsf_grid.best_score_:.4f}")
print(f"   Mejores parámetros:")
for k, v in rsf_grid.best_params_.items():
    print(f"      {k}: {v}")

print(f"\n🚀 XGBOOST-AFT:")
print(f"   Loss (aft-nloglik): {best_loss_val:.4f}")
print(f"   Distribución ganadora: {best_params.get('aft_loss_distribution', 'N/A')}")
print(f"   Mejor num_boost_round: {best_num_rounds}")

print(f"\n📁 MODELOS GUARDADOS:")
print(f"   - models/best_rsf.pkl")
print(f"   - models/best_xgb_aft.json")

print("\n" + "=" * 70)
print("✅ PROCESO COMPLETADO. Listo para evaluación final en test set.")
print("=" * 70)